In [30]:
import boto3

## Reset every staging tier except for Landing

In [24]:
!aws s3 rm s3://project-lf/etl/raw --recursive
!aws s3 rm s3://project-lf/etl/access --recursive
!aws s3 rm s3://project-lf/etl/optimised --recursive

delete: s3://project-lf/etl/raw/claim_db/claim/full/202306022049/part-00000-27a8b021-6da7-4b8d-a14c-3c234baaed99-c000.snappy.parquet
delete: s3://project-lf/etl/raw/claim_db/policyholder/full/202306022106/part-00000-df95f6cc-aee4-4573-bcad-39dd5ffd1428-c000.snappy.parquet
delete: s3://project-lf/etl/raw/claim_db/provider/full/202306020108/part-00000-a369051d-e5fc-42d2-93b6-cec2238eb804-c000.snappy.parquet
delete: s3://project-lf/etl/raw/claim_db/claim/full/202306020108/part-00000-b6fbd60f-8a52-4f92-b8c7-8699e5c91912-c000.snappy.parquet
delete: s3://project-lf/etl/raw/claim_db/provider/full/202306021618/part-00000-a8b75da5-7fdd-4cbb-a31a-1a92ffc628c5-c000.snappy.parquet
delete: s3://project-lf/etl/raw/claim_db/provider/full/202306022105/part-00000-a9b88f44-2248-4d91-8327-c5122571239e-c000.snappy.parquet
delete: s3://project-lf/etl/raw/claim_db/policyholder/full/202306020108/part-00000-3cfb6d4a-0dda-4701-8577-dbb02b998dc4-c000.snappy.parquet
delete: s3://project-lf/etl/raw/claim_db/provi

## Load source data into `Bucket.TEST` and `Bucket.PROD` if needed

In [4]:
!aws s3 ls s3://project-lf/etl/landing --recursive

2023-06-03 00:21:47      22655 etl/landing/claim_db/claim/full/202305211851-claim-full.csv
2023-06-03 00:21:51       5818 etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv
2023-06-03 00:21:55        814 etl/landing/claim_db/provider/full/202305221136-provider-full.csv


In [71]:
!aws s3 cp /home/glue_user/project_lf/ETL-TDD/etl/mock/source_files/202305211851-claim-full.csv s3://project-lf/etl/landing/claim_db/claim/full/202305211851-claim-full.csv

!aws s3 cp /home/glue_user/project_lf/ETL-TDD/etl/mock/source_files/202305221132-policyholder-full.csv s3://project-lf/etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv

!aws s3 cp /home/glue_user/project_lf/ETL-TDD/etl/mock/source_files/202305221136-provider-full.csv s3://project-lf/etl/landing/claim_db/provider/full/202305221136-provider-full.csv

upload: ../etl/mock/source_files/202305211851-claim-full.csv to s3://project-lf/etl/landing/claim_db/claim/full/202305211851-claim-full.csv
upload: ../etl/mock/source_files/202305221132-policyholder-full.csv to s3://project-lf/etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv
upload: ../etl/mock/source_files/202305221136-provider-full.csv to s3://project-lf/etl/landing/claim_db/provider/full/202305221136-provider-full.csv


In [88]:
env = Bucket.TEST

files = {
    "/home/glue_user/project_lf/ETL-TDD/etl/mock/source_files/202305211851-claim-full.csv": f"s3://{env}/etl/landing/claim_db/claim/full/202305211851-claim-full.csv",
    "/home/glue_user/project_lf/ETL-TDD/etl/mock/source_files/202305221132-policyholder-full.csv": f"s3://{env}/etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv",
    "/home/glue_user/project_lf/ETL-TDD/etl/mock/source_files/202305221136-provider-full.csv": f"s3://{env}/etl/landing/claim_db/provider/full/202305221136-provider-full.csv",
}

# s3_resource = S3Resource.getInstance()

for src, dest in files.items():

    print(bucket_name, key)

test-lf-wm etl/landing/claim_db/claim/full/202305211851-claim-full.csv
test-lf-wm etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv
test-lf-wm etl/landing/claim_db/provider/full/202305221136-provider-full.csv


## Delete every glue job

In [87]:
# delete all glue ETL jobs
glue = boto3.client('glue', region_name='us-east-1')
jobs = glue.get_jobs()
for job in jobs['Jobs']:
    if job['Name'].startswith('stage_'):
        glue.delete_job(JobName=job['Name'])

## Refresh every .py job file in `code/jobs/` and `code/dependencies` 

In [ ]:
# delete the libraries.zip file at s3://project-lf/code/libraries.zip
!aws s3 rm s3://project-lf/code/libraries.zip

# create a new zip file
!zip -r libraries.zip *

# write cli that copies libraries.zip to s3://project-lf/code/libraries.zip
!aws s3 cp libraries.zip s3://project-lf/code/dependencies/libraries.zip

# delete everything but the libraries.zip file at s3://project-lf/code/
!aws s3 rm s3://project-lf/code/ --recursive --exclude "libraries.zip"

In [98]:
# write cli that copies libraries.zip to s3://project-lf/code/libraries.zip
!aws s3 cp libraries.zip s3://project-lf/code/dependencies/libraries.zip


upload: ./libraries.zip to s3://project-lf/code/dependencies/libraries.zip


## Upload .py job files to S3

In [84]:
import boto3

# Create an S3 client
s3_client = boto3.client('s3')

# Set the bucket and prefix
bucket = 'project-lf'
prefix = 'code/jobs/'

# Set the list of files to upload
files = [
    '/home/glue_user/project_lf/ETL-TDD/stage_claim_into_access.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_claim_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_claim_into_raw.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_date_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_location_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_policyholder_into_access.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_policyholder_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_policyholder_into_raw.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_procedure_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_provider_into_access.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_provider_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_provider_into_raw.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_source_into_landing.py'
]

# Upload each file to the S3 bucket
for file_path in files:
    # Get the file name
    file_name = file_path.split('/')[-1]

    # Set the object key
    key = f'{prefix}{file_name}'

    # Upload the file
    s3_client.upload_file(file_path, bucket, key)

print(f'Uploaded {len(files)} files to s3://{bucket}/{prefix}')

Uploaded 13 files to s3://project-lf/code/jobs/


## Initialise Glue session

In [77]:
# for each script in s3://project-lf/code/jobs/ create a glue job
import boto3

# Create an AWS Glue client
glue_client = boto3.client('glue', region_name='us-east-1')

# Set the parameters for the new Glue job
glue_role = 'data-quality-lf'


## Get a list of all script objects in `project-lf/code/jobs/`

In [86]:
# get all keys in s3://project-lf/code/jobs/
s3_resource = boto3.resource('s3')
bucket = 'project-lf'
prefix = 'code/jobs/'
objects = s3_resource.Bucket(bucket).objects.filter(Prefix=prefix)
list(objects)

[s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_claim_into_access.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_claim_into_optimised.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_claim_into_raw.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_date_into_optimised.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_location_into_optimised.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_policyholder_into_access.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_policyholder_into_optimised.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_policyholder_into_raw.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_procedure_into_optimised.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_provider_into_access.py'),
 s3.ObjectSummary(bucket_name='project-lf', key='code/jobs/stage_provid

## Create a job for every script object

In [88]:
glue_client = boto3.client('glue', region_name='us-east-1')
job_names = []
for s3_object in objects:
    print(s3_object.key)
    
    # Create an AWS Glue client

    # Set the parameters for the new Glue job
    glue_role = 'data-quality-lf'
    script_location = f's3://project-lf/{s3_object.key}'
    job_name = script_location.split('/')[-1].split('.')[0]
    extra_py_files = 's3://project-lf/code/dependencies/libraries.zip'
    glue_version = '4.0'
    additional_python_modules = "holidays"

    try:
        # Create the new Glue job
        response = glue_client.create_job(
            Name=job_name,
            Role=glue_role,
            Command={'Name': 'glueetl', 'ScriptLocation': script_location},
            GlueVersion=glue_version,
            DefaultArguments={
                '--extra-py-files': extra_py_files,
                '--additional-python-modules': additional_python_modules
            },
            WorkerType='G.1X',
            NumberOfWorkers=2
        )
        print(response)
    except Exception as e:
        print("Skipping job creation for", job_name, e)
    
    job_names.append(job_name)

code/jobs/stage_claim_into_access.py
{'Name': 'stage_claim_into_access', 'ResponseMetadata': {'RequestId': 'a8d51c8f-04f6-4376-952d-615c1a7af271', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 01 Jun 2023 14:55:34 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '34', 'connection': 'keep-alive', 'x-amzn-requestid': 'a8d51c8f-04f6-4376-952d-615c1a7af271'}, 'RetryAttempts': 0}}
code/jobs/stage_claim_into_optimised.py
{'Name': 'stage_claim_into_optimised', 'ResponseMetadata': {'RequestId': 'b3365302-8774-4f81-a2b7-86a8f77b9afe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 01 Jun 2023 14:55:35 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '37', 'connection': 'keep-alive', 'x-amzn-requestid': 'b3365302-8774-4f81-a2b7-86a8f77b9afe'}, 'RetryAttempts': 0}}
code/jobs/stage_claim_into_raw.py
{'Name': 'stage_claim_into_raw', 'ResponseMetadata': {'RequestId': '4705c06e-779c-4e6f-bad6-2573f07ce73d', 'HTTPStatusCode': 200, 'HTTPHeaders': {

## Define execution order of jobs

In [89]:
job_waves = [
     ['stage_claim_into_raw', 'stage_provider_into_raw', 'stage_policyholder_into_raw',
    'stage_date_into_optimised'],
     
    ['stage_claim_into_access','stage_provider_into_access',
        'stage_policyholder_into_access'],
    
    ['stage_location_into_optimised', 'stage_procedure_into_optimised'],
    
    ['stage_policyholder_into_optimised', 'stage_provider_into_optimised'],
    
    ['stage_claim_into_optimised']
]

## Define a function to run (in parallel) all jobs in a list and wait until they have completed

In [90]:
import boto3
import time
import colorama
from colorama import Fore, Style
colorama.init()

# Create an AWS Glue client
glue_client = boto3.client('glue', region_name='us-east-1')

# Function to run a list of jobs and wait for them to finish
def run_jobs(jobs):
    # Start each job and store its job run ID
    job_run_ids = {}
    for job_name in jobs:
        response = glue_client.start_job_run(JobName=job_name)
        job_run_id = response['JobRunId']
        job_run_ids[job_name] = job_run_id
        print(job_name, job_run_id)

    print("-"*80)
    
    # Wait for all jobs to finish
    while True:
        # Check the status of each job run
        all_finished = True
    
        for job_name, job_run_id in job_run_ids.items():
            try:
                response = glue_client.get_job(JobName=job_name)
                response = glue_client.get_job_run(JobName=job_name, RunId=job_run_id)
                status = response['JobRun']['JobRunState']
                if status not in ['SUCCEEDED', 'FAILED', 'STOPPED']:
                    all_finished = False
                else:
                    if status == 'SUCCEEDED':
                        print(Fore.GREEN + '\t' + job_name + ' has ' + status + '.' + Style.RESET_ALL)
                    elif status == 'FAILED':
                        error_message = response['JobRun'].get('ErrorMessage', 'No error message available')
                        print(Fore.RED + '\t' + job_name + ' has ' + status + '.' + Style.RESET_ALL)
                        print(Fore.RED + f'\t{job_name} has {status}. Error message: {error_message}' + Style.RESET_ALL)
                    else:
                        print(Fore.YELLOW + f'\t{job_name} has {status}.' + Style.RESET_ALL)
            except glue_client.exceptions.EntityNotFoundException:
                continue
        print("-"*80)
        # If all jobs have finished, exit the loop
        if all_finished:
            break
        # Otherwise, wait and check again
        time.sleep(10)

## Execute jobs in order

In [100]:
for count, job_set in enumerate(job_waves):
    print("*"*80)
    print(f"Starting on Run {count + 1}: {job_set}")
    print("*"*80)
    run_jobs(job_set)
print('All jobs finished')

********************************************************************************
Starting on Run 1: ['stage_claim_into_raw', 'stage_provider_into_raw', 'stage_policyholder_into_raw', 'stage_date_into_optimised']
********************************************************************************
stage_claim_into_raw jr_f013cdcd5ba3156e87729f26f19dc25d5ebcffcf263ab4858c02f2b56e4d5776
stage_provider_into_raw jr_06ab19d466f1cb9a05dd8fe770393be13bebf127b9434129619e81282a2d5166
stage_policyholder_into_raw jr_fdc34733a113a4eaba4bb9a66582be5bf7208a1807792cd3f7bcc72d1b389858
stage_date_into_optimised jr_0a425aa1c16043e5e8f600a973c86f7a4d4890574cd22b8b1a073d1428696990
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
------------

# Experiment with boto3

In [11]:
import boto3

def stage_file(bucket_name, file_key, staging_bucket):
    s3_client = boto3.client('s3')

    # Copy the file from the source bucket to the staging bucket
    s3_client.copy_object(
        Bucket=staging_bucket,
        CopySource={'Bucket': bucket_name, 'Key': file_key},
        Key=file_key
    )

def run_crawler(crawler_name):
    glue_client = boto3.client('glue')

    # Start the Glue crawler
    response = glue_client.start_crawler(Name=crawler_name)

    # Wait for the crawler to complete
    glue_client.get_waiter('crawler_complete').wait(
        Name=crawler_name,
        WaiterConfig={'Delay': 2, 'MaxAttempts': 100}
    )

def run_databrew_profile_job(job_name, dataset_name):
    databrew_client = boto3.client('databrew')

    # Create a new AWS Glue DataBrew profile job run
    response = databrew_client.create_job_run(
        Name=job_name,
        Type='PROFILE',
        DatasetName=dataset_name
    )

    # Get the job run ID
    job_run_id = response['JobRunId']
    print(f"Started DataBrew profile job run. Job Run ID: {job_run_id}")

    # Wait for the job run to complete
    databrew_client.get_waiter('job_run_state_change').wait(
        Name=job_name,
        RunId=job_run_id,
        WaiterConfig={'Delay': 10, 'MaxAttempts': 60}
    )

    print("DataBrew profile job run completed.")

# Example usage
bucket_name = 'your-source-bucket'
file_key = 'path/to/file.csv'
staging_bucket = 'your-staging-bucket'
crawler_name = 'your-crawler-name'
databrew_job_name = 'your-databrew-job-name'
dataset_name = 'your-dataset-name'

# Stage the file
stage_file(bucket_name, file_key, staging_bucket)

# Run the Glue crawler
run_crawler(crawler_name)

# Run the DataBrew profile job on the staged file
run_databrew_profile_job(databrew_job_name, dataset_name)


ClientError: An error occurred (AccessDenied) when calling the CopyObject operation: Access Denied

In [22]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0

"""
Purpose

Shows how to use the AWS SDK for Python (Boto3) with AWS Glue to
create and manage crawlers, databases, and jobs.
"""

import logging
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)


# snippet-start:[python.example_code.glue.GlueWrapper.full]
# snippet-start:[python.example_code.glue.GlueWrapper.decl]
class GlueWrapper:
    """Encapsulates AWS Glue actions."""
    def __init__(self, glue_client):
        """
        :param glue_client: A Boto3 Glue client.
        """
        self.glue_client = glue_client
# snippet-end:[python.example_code.glue.GlueWrapper.decl]

    # snippet-start:[python.example_code.glue.GetCrawler]
    def get_crawler(self, name):
        """
        Gets information about a crawler.

        :param name: The name of the crawler to look up.
        :return: Data about the crawler.
        """
        crawler = None
        try:
            response = self.glue_client.get_crawler(Name=name)
            crawler = response['Crawler']
        except ClientError as err:
            if err.response['Error']['Code'] == 'EntityNotFoundException':
                logger.info("Crawler %s doesn't exist.", name)
            else:
                logger.error(
                    "Couldn't get crawler %s. Here's why: %s: %s", name,
                    err.response['Error']['Code'], err.response['Error']['Message'])
                raise
        return crawler
    # snippet-end:[python.example_code.glue.GetCrawler]

    # snippet-start:[python.example_code.glue.CreateCrawler]
    def create_crawler(self, name, role_arn, db_name, db_prefix, s3_target):
        """
        Creates a crawler that can crawl the specified target and populate a
        database in your AWS Glue Data Catalog with metadata that describes the data
        in the target.

        :param name: The name of the crawler.
        :param role_arn: The Amazon Resource Name (ARN) of an AWS Identity and Access
                         Management (IAM) role that grants permission to let AWS Glue
                         access the resources it needs.
        :param db_name: The name to give the database that is created by the crawler.
        :param db_prefix: The prefix to give any database tables that are created by
                          the crawler.
        :param s3_target: The URL to an S3 bucket that contains data that is
                          the target of the crawler.
        """
        try:
            self.glue_client.create_crawler(
                Name=name,
                Role=role_arn,
                DatabaseName=db_name,
                TablePrefix=db_prefix,
                Targets={'S3Targets': [{'Path': s3_target}]})
        except ClientError as err:
            logger.error(
                "Couldn't create crawler. Here's why: %s: %s",
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
    # snippet-end:[python.example_code.glue.CreateCrawler]

    # snippet-start:[python.example_code.glue.StartCrawler]
    def start_crawler(self, name):
        """
        Starts a crawler. The crawler crawls its configured target and creates
        metadata that describes the data it finds in the target data source.

        :param name: The name of the crawler to start.
        """
        try:
            self.glue_client.start_crawler(Name=name)
        except ClientError as err:
            logger.error(
                "Couldn't start crawler %s. Here's why: %s: %s", name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
    # snippet-end:[python.example_code.glue.StartCrawler]

    # snippet-start:[python.example_code.glue.GetDatabase]
    def get_database(self, name):
        """
        Gets information about a database in your Data Catalog.

        :param name: The name of the database to look up.
        :return: Information about the database.
        """
        try:
            response = self.glue_client.get_database(Name=name)
        except ClientError as err:
            logger.error(
                "Couldn't get database %s. Here's why: %s: %s", name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
        else:
            return response['Database']
    # snippet-end:[python.example_code.glue.GetDatabase]

    # snippet-start:[python.example_code.glue.GetTables]
    def get_tables(self, db_name):
        """
        Gets a list of tables in a Data Catalog database.

        :param db_name: The name of the database to query.
        :return: The list of tables in the database.
        """
        try:
            response = self.glue_client.get_tables(DatabaseName=db_name)
        except ClientError as err:
            logger.error(
                "Couldn't get tables %s. Here's why: %s: %s", db_name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
        else:
            return response['TableList']
    # snippet-end:[python.example_code.glue.GetTables]

    # snippet-start:[python.example_code.glue.CreateJob]
    def create_job(self, name, description, role_arn, script_location):
        """
        Creates a job definition for an extract, transform, and load (ETL) job that can
        be run by AWS Glue.

        :param name: The name of the job definition.
        :param description: The description of the job definition.
        :param role_arn: The ARN of an IAM role that grants AWS Glue the permissions
                         it requires to run the job.
        :param script_location: The Amazon S3 URL of a Python ETL script that is run as
                                part of the job. The script defines how the data is
                                transformed.
        """
        try:
            self.glue_client.create_job(
                Name=name, Description=description, Role=role_arn,
                Command={'Name': 'glueetl', 'ScriptLocation': script_location, 'PythonVersion': '3'},
                GlueVersion='3.0')
        except ClientError as err:
            logger.error(
                "Couldn't create job %s. Here's why: %s: %s", name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
    # snippet-end:[python.example_code.glue.CreateJob]

    # snippet-start:[python.example_code.glue.StartJobRun]
    def start_job_run(self, name, input_database, input_table, output_bucket_name):
        """
        Starts a job run. A job run extracts data from the source, transforms it,
        and loads it to the output bucket.

        :param name: The name of the job definition.
        :param input_database: The name of the metadata database that contains tables
                               that describe the source data. This is typically created
                               by a crawler.
        :param input_table: The name of the table in the metadata database that
                            describes the source data.
        :param output_bucket_name: The S3 bucket where the output is written.
        :return: The ID of the job run.
        """
        try:
            # The custom Arguments that are passed to this function are used by the
            # Python ETL script to determine the location of input and output data.
            response = self.glue_client.start_job_run(
                JobName=name,
                Arguments={
                    '--input_database': input_database,
                    '--input_table': input_table,
                    '--output_bucket_url': f's3://{output_bucket_name}/'})
        except ClientError as err:
            logger.error(
                "Couldn't start job run %s. Here's why: %s: %s", name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
        else:
            return response['JobRunId']
    # snippet-end:[python.example_code.glue.StartJobRun]

    # snippet-start:[python.example_code.glue.ListJobs]
    def list_jobs(self):
        """
        Lists the names of job definitions in your account.

        :return: The list of job definition names.
        """
        try:
            response = self.glue_client.list_jobs()
        except ClientError as err:
            logger.error(
                "Couldn't list jobs. Here's why: %s: %s",
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
        else:
            return response['JobNames']
    # snippet-end:[python.example_code.glue.ListJobs]

    # snippet-start:[python.example_code.glue.GetJobRuns]
    def get_job_runs(self, job_name):
        """
        Gets information about runs that have been performed for a specific job
        definition.

        :param job_name: The name of the job definition to look up.
        :return: The list of job runs.
        """
        try:
            response = self.glue_client.get_job_runs(JobName=job_name)
        except ClientError as err:
            logger.error(
                "Couldn't get job runs for %s. Here's why: %s: %s", job_name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
        else:
            return response['JobRuns']
    # snippet-end:[python.example_code.glue.GetJobRuns]

    # snippet-start:[python.example_code.glue.GetJobRun]
    def get_job_run(self, name, run_id):
        """
        Gets information about a single job run.

        :param name: The name of the job definition for the run.
        :param run_id: The ID of the run.
        :return: Information about the run.
        """
        try:
            response = self.glue_client.get_job_run(JobName=name, RunId=run_id)
        except ClientError as err:
            logger.error(
                "Couldn't get job run %s/%s. Here's why: %s: %s", name, run_id,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
        else:
            return response['JobRun']
    # snippet-end:[python.example_code.glue.GetJobRun]

    # snippet-start:[python.example_code.glue.DeleteJob]
    def delete_job(self, job_name):
        """
        Deletes a job definition. This also deletes data about all runs that are
        associated with this job definition.

        :param job_name: The name of the job definition to delete.
        """
        try:
            self.glue_client.delete_job(JobName=job_name)
        except ClientError as err:
            logger.error(
                "Couldn't delete job %s. Here's why: %s: %s", job_name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
    # snippet-end:[python.example_code.glue.DeleteJob]

    # snippet-start:[python.example_code.glue.DeleteTable]
    def delete_table(self, db_name, table_name):
        """
        Deletes a table from a metadata database.

        :param db_name: The name of the database that contains the table.
        :param table_name: The name of the table to delete.
        """
        try:
            self.glue_client.delete_table(DatabaseName=db_name, Name=table_name)
        except ClientError as err:
            logger.error(
                "Couldn't delete table %s. Here's why: %s: %s", table_name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
    # snippet-end:[python.example_code.glue.DeleteTable]

    # snippet-start:[python.example_code.glue.DeleteDatabase]
    def delete_database(self, name):
        """
        Deletes a metadata database from your Data Catalog.

        :param name: The name of the database to delete.
        """
        try:
            self.glue_client.delete_database(Name=name)
        except ClientError as err:
            logger.error(
                "Couldn't delete database %s. Here's why: %s: %s", name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
    # snippet-end:[python.example_code.glue.DeleteDatabase]

    # snippet-start:[python.example_code.glue.DeleteCrawler]
    def delete_crawler(self, name):
        """
        Deletes a crawler.

        :param name: The name of the crawler to delete.
        """
        try:
            self.glue_client.delete_crawler(Name=name)
        except ClientError as err:
            logger.error(
                "Couldn't delete crawler %s. Here's why: %s: %s", name,
                err.response['Error']['Code'], err.response['Error']['Message'])
            raise
    # snippet-end:[python.example_code.glue.DeleteCrawler]
# snippet-end:[python.example_code.glue.GlueWrapper.full]

## Create Crawler
https://docs.aws.amazon.com/glue/latest/dg/example_glue_CreateCrawler_section.html

In [37]:
import boto3

role = "arn:aws:iam::618572314333:role/service-role/AWSGlueServiceRole-ETL"
name = "boto3-dq-crawler"
database_name = "data-quality"
s3_target_path = "s3://project-lf/data_quality"

glue = GlueWrapper(boto3.client('glue', region_name='us-east-1'))

try:
    response = glue.create_crawler(name, role, database_name, '', s3_target_path)
    print(response)
except glue.glue_client.exceptions.AlreadyExistsException as e:
    print(e)
    
if glue.get_crawler(name) == None:
    print("Crawler does not exist.")
else:
    print("Crawler exists.")
    
try:
    response = glue.start_crawler(name)
    print(response)
except glue.glue_client.exceptions.CrawlerRunningException as e:
    print(e)

Couldn't create crawler. Here's why: AlreadyExistsException: 618572314333:boto3-dq-crawler already exists


An error occurred (AlreadyExistsException) when calling the CreateCrawler operation: 618572314333:boto3-dq-crawler already exists
Crawler exists.


Couldn't start crawler boto3-dq-crawler. Here's why: CrawlerRunningException: Crawler with name boto3-dq-crawler has already started


An error occurred (CrawlerRunningException) when calling the StartCrawler operation: Crawler with name boto3-dq-crawler has already started


In [35]:
response = glue.get_crawler(name)
print(response)

{'Name': 'boto3-dq-crawler', 'Role': 'service-role/AWSGlueServiceRole-ETL', 'Targets': {'S3Targets': [{'Path': 's3://project-lf/data_quality', 'Exclusions': []}], 'JdbcTargets': [], 'MongoDBTargets': [], 'DynamoDBTargets': [], 'CatalogTargets': [], 'DeltaTargets': []}, 'DatabaseName': 'data-quality', 'Classifiers': [], 'RecrawlPolicy': {'RecrawlBehavior': 'CRAWL_EVERYTHING'}, 'SchemaChangePolicy': {'UpdateBehavior': 'UPDATE_IN_DATABASE', 'DeleteBehavior': 'DEPRECATE_IN_DATABASE'}, 'LineageConfiguration': {'CrawlerLineageSettings': 'DISABLE'}, 'State': 'STOPPING', 'TablePrefix': 'aws_glue_', 'CrawlElapsedTime': 46000, 'CreationTime': datetime.datetime(2023, 6, 4, 0, 51, 11, tzinfo=tzlocal()), 'LastUpdated': datetime.datetime(2023, 6, 4, 0, 51, 11, tzinfo=tzlocal()), 'Version': 1, 'LakeFormationConfiguration': {'UseLakeFormationCredentials': False, 'AccountId': ''}}


In [20]:
create_crawler(name, role, database_name, s3_target_path)

{'ResponseMetadata': {'RequestId': '06c3ae2f-bb8b-4c49-bb95-b22564c3fe3d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 04 Jun 2023 00:51:47 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '06c3ae2f-bb8b-4c49-bb95-b22564c3fe3d'},
  'RetryAttempts': 0}}

### Create and execute landing tier crawler

In [43]:
role = "arn:aws:iam::618572314333:role/service-role/AWSGlueServiceRole-ETL"
name = "boto3-landing-production"
database_name = "release-3"
prefix = "landing"
s3_target_path = "s3://project-lf/etl/landing"

In [41]:
client = GlueWrapper(boto3.client('glue', region_name='us-east-1'))

In [71]:
client.create_crawler(name, role, database_name, '', s3_target_path)

In [72]:
client.start_crawler(name)

In [70]:
role = "arn:aws:iam::618572314333:role/service-role/AWSGlueServiceRole-ETL"
name = "test-boto3-landing-production"
database_name = "release-3"
prefix = "landing"
s3_target_path = "s3://test-lf-wm/etl/landing"

## Create and run profile jobs of landing tier

In [48]:
databrew = boto3.client('databrew', region_name='us-east-1')

# Ideas

## Wrapper for dq report

In [50]:
import json

def get_dq_results(bucket, key, s3_resource):
    """
    Get the data quality results from the S3 bucket.
    """
    s3_object = s3_resource.Object(bucket, key)
    dq_results = s3_object.get()["Body"].read().decode("utf-8")
    dict_results = json.loads(dq_results)

    return dict_results

In [ ]:
bucket = 'project-lf'
key = 'dataquality/landing-claimdb-policyholder_45af5c6917a253505482214c95eb38d832948c5aa6c3e80bf4efb71426938ea8.json'

s3 = boto3.resource('s3', region_name='us-east-1')
get_dq_results(bucket, key, s3)

In [ ]:
get_dq_results(bucket, key, s3)

In [99]:
import boto3

databrew = boto3.client('databrew')

response = databrew.create_profile_job(
    DatasetName='landing-claimdb-provider',
    Name='production-landing-claimdb-provider',
    OutputLocation={
        'Bucket': 'project-lf',
        'Key': 'dataquality/'
    },
    RoleArn='arn:aws:iam::618572314333:role/DataBrew-Data-Quality-Workflows-DataAccessRole-ZRAZ5RGWCQW0'
)

print(response)

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the CreateProfileJob operation: Dataset landing-claimdb-provider wasn't found

In [ ]:
import boto3

databrew = boto3.client('databrew', region_name='us-east-1')
bucket = 'test-lf-wm'
key = 'etl/landing/claim_db/provider/full/202305221136-provider-full.csv'
dataset_name = 'test-landing-claimdb-provider'

response = databrew.create_dataset(
    Name="202305221132-policyholder-full",
    Format='GLUE_TABLE',
    Input={
        'DatabaseInputDefinition': {
            'DatabaseName': 'test-release-3',
            'TableName': dataset_name
        }
    }
) 


print(response)


# ConflictException if dataset already exists.

## Start Run 1

### Load data.
Inputs needed: 

Process: `stage_x_into_y`

Output: `df` and `full path`

### Create data catalog.
Inputs needed: 

Process: 

Output: `df` and `full path`

### Connect dataset to data catalog.
Inputs needed: 

Process: 

Output: `df` and `full path`

### Run profile job.
Inputs needed: 

Process: 

Output: `df` and `full path`

### Get profile job results.
Inputs needed: 

Process: 

Output: `df` and `full path`

### Create data quality job.
Inputs needed: 

Process: 

Output: `df` and `full path`

### Run data quality job.
Inputs needed: 

Process: 

Output: `df` and `full path`

### Get data quality job results.
Inputs needed: 

Process: 

Output: `df` and `full path`


## Start Run 2

In [3]:
import sys

sys.path.append('/home/glue_user/project_lf/ETL-TDD')

In [110]:
from automation.databrew import DataBrew

databrew = DataBrew('us-east-1')

In [4]:
import stage_claim_into_raw
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("test").getOrCreate()
df, path = stage_claim_into_raw.run(spark, 'test-lf-wm')

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
log4j:WARN No appenders could be found for logger (org.apache.hadoop.metrics2.lib.MutableMetricsFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


getInstance(**kwargs={}) called
kwargs is {'region_name': 'us-east-1'}
endpoint is s3(https://s3.amazonaws.com)


In [132]:
pathO

's3://test-lf-wm/etl/raw/claim_db/claim/full/202306041621/'

In [9]:
def get_result(path):
    """
    Parse S3 path into bucket and key.
    """
    path = path.lower().strip()
    components = path.split('/')
    return {
        'path' : path,
        'bucket' : components[2],
        'tier' : components[4],
        'table' : components[6],
        'key' : '/'.join(components[4:][:-1]),
        'env': components[2].split('-')[0]
    }
    
pathO = path

In [55]:
# ----------------------------------------------------------------------------
# GLUE AUTOMATION UTILITIES
# ----------------------------------------------------------------------------
def get_job_name(path):
    """
    Get the job name for the DataBrew profile job.
    """
    return f"{path['env']}-{path['tier']}-{path['table']}"

def get_table_name(path):
    """
    Get the table name for the DataBrew profile job.
    """
    return f"{path['tier']}{path['table']}"

In [56]:
get_job_name(path), get_table_name(path)

('test-raw-claim', 'rawclaim')

In [115]:
# ----------------------------------------------------------------------------
# CREATE PROFILE JOB
# ----------------------------------------------------------------------------
try: 
    databrew.create_profile_job(get_job_name(path),
                            f"profile-{get_job_name(path)}", 
                            bucket='test-lf-wm', 
                            key='dataquality')
except databrew.client.exceptions.ConflictException:
    print(f"Job 'profile-{get_job_name(path)}' already exists")

Job 'profile-test-raw-claim' already exists


In [116]:
# ----------------------------------------------------------------------------
# CREATE DATASET FOR DATA CATALOG ITEM
# ----------------------------------------------------------------------------
try:
    job = get_job_name(path)
    table = get_table_name(path)
    databrew.create_dataset(name=job, 
                        table=table, 
                        database='test-release-3', 
                        CatalogId='618572314333')
except databrew.client.exceptions.ConflictException:
    print(f"Dataset '{name}' already exists.")

Dataset 'test-raw' already exists.


In [121]:
# ----------------------------------------------------------------------------
# RUN PROFILE JOB
# ----------------------------------------------------------------------------
try:
    databrew.start_profile_job(f"profile-{get_job_name(path)}")
except databrew.client.exceptions.ConflictException:
    print(f"Job 'profile-{get_job_name(path)}' is already running")
except databrew.client.exceptions.ResourceNotFoundException:
    print(f"Job 'profile-{get_job_name(path)}' does not exist")

Job 'profile-test-raw-claim' is already running


In [124]:
def get_profile_object_path(job_name):
    """
    Get the profile object for a given job name.
    """
    latest_run = databrew.client.list_job_runs(Name=job_name)['JobRuns'][0]
    # Get location information.
    location = latest_run['Outputs'][0]['Location']
    # Get path
    bucket, key = location['Bucket'], location['Key']
    return bucket,key

In [101]:
def get_profile_link(dataset):
    return f"https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset={dataset}&tab=profile-overview"

In [127]:
get_profile_object_path(job_name=f"profile-test-raw-claim")

('test-lf-wm',
 'dataquality/test-raw-claim_672988221f4998bda2c49474bb323ba1898599bb7895830747936916f9269bb3.json')

In [78]:
for job in databrew.client.list_job_runs(Name=f"profile-{get_job_name(path)}")['JobRuns']:
    finished_running = True
    if job['State'] != 'SUCCEEDED':
        finished_running = False
    print(json.dumps(job, indent=4,default=str))
    print(job['State'])
    print(get_result(job['Outputs']['Location']['Key']))

{
    "Attempt": 0,
    "CompletedOn": "2023-06-04 08:00:20.610000+00:00",
    "DatasetName": "test-raw-claim",
    "ExecutionTime": 135,
    "JobName": "profile-test-raw-claim",
    "RunId": "db_e8f23d26addbdb1da6258f6a086e99bd3724deded967ad1deef4682849b04386",
    "State": "SUCCEEDED",
    "LogSubscription": "DISABLE",
    "Outputs": [
        {
            "Format": "JSON",
            "Location": {
                "Bucket": "test-lf-wm",
                "Key": "dataquality/test-raw-claim_e8f23d26addbdb1da6258f6a086e99bd3724deded967ad1deef4682849b04386.json",
                "BucketOwner": "618572314333"
            },
            "Overwrite": false
        }
    ],
    "StartedBy": "arn:aws:iam::618572314333:user/Lead",
    "StartedOn": "2023-06-04 07:57:54.526000+00:00",
    "JobSample": {
        "Mode": "CUSTOM_ROWS",
        "Size": 20000
    }
}
SUCCEEDED


In [107]:
# Fetch the job results
results = databrew.client.describe_profile_job_run

AttributeError: 'GlueDataBrew' object has no attribute 'describe_profile_job_run'

In [ ]:
import boto3

# Create a CloudWatch Logs client
cloudwatch_logs = boto3.client('logs', region_name='us-east-1')

# Get the log events
response = cloudwatch_logs.get_log_events(
    logGroupName='/aws-glue/crawlers',
    logStreamName='test-raw',
)

# Print the log events
for event in response['events']:
    print(event['timestamp'], event['message'])


In [134]:
write_path = 's3://test-lf-wm/etl/raw/claim_db/claim/full/202306041621/'

# extract up to claim_db
path = write_path.split('/')
path = '/'.join(path[:path.index('claim_db')])
path

's3://test-lf-wm/etl/raw'

In [28]:
from databrew import DataBrew
from glue import GlueWrapper
from time import time, sleep 

glue = GlueWrapper('us-east-1')
databrew = DataBrew('us-east-1')

path = 's3://test-lf-wm/etl/raw/claim_db/claim/full/202306041621/'

# Create crawler to create catalog.
role = "arn:aws:iam::618572314333:role/service-role/AWSGlueServiceRole-ETL"
name = f"{path['env']}-{path['tier']}"
database_name = f"{path['env']}-release-3"
s3_target_path = glue.create_crawler_path(path)


# ----------------------------------------------------------------------------
# CREATE AND START CRAWLER
# ----------------------------------------------------------------------------
try:
    glue.create_crawler(name, role, database_name, path['tier'], s3_target_path)
except glue.glue_client.exceptions.AlreadyExistsException:
    print(f"Crawler {name} already exists.")

try:
    # glue.start_crawler(name)
    print("Pretending crawler was started.")
    print(f"Crawler {name} started.")
except glue.glue_client.exceptions.CrawlerRunningException:
    print(f"Crawler {name} is already running.")
    
# glue.start_crawler(name)
# wait for crawler to finish by checking its reponse
start = time()
while glue.get_crawler(name)['State'] not in ['READY']:
    print(f"Crawler {name} is {glue.get_crawler(name)['State']}.")
    sleep(10)
    print('\t', int(time() - start))
    if int(time() - start) > 60 * 3:
        print("Crawler wait period timed out.")
        break
finish = time()

print(f"Time to view result: {finish - start}")

# ----------------------------------------------------------------------------
# CREATE A DATASET CONNECTED TO THE DATA CATALOG
# ----------------------------------------------------------------------------
try:
    job = get_job_name(path)
    table = get_table_name(path)
    databrew.create_dataset(name=job, 
                        table=table, 
                        database='test-release-3', 
                        CatalogId='618572314333')
except databrew.client.exceptions.ConflictException:
    print(f"Dataset '{name}' already exists.")

# ----------------------------------------------------------------------------
# CREATE A PROFILE JOB FOR THE DATASET
# ----------------------------------------------------------------------------
try: 
    databrew.create_profile_job(get_job_name(path),
                            f"profile-{get_job_name(path)}", 
                            bucket='test-lf-wm', 
                            key='dataquality')
except databrew.client.exceptions.ConflictException:
    print(f"Job 'profile-{get_job_name(path)}' already exists")


# ----------------------------------------------------------------------------
# RUN PROFILE JOB
# ----------------------------------------------------------------------------
try:
    databrew.start_profile_job(f"profile-{get_job_name(path)}")
except databrew.client.exceptions.ConflictException:
    print(f"Job 'profile-{get_job_name(path)}' is already running")

# ----------------------------------------------------------------------------
# GET PROFILE PATH
# ----------------------------------------------------------------------------
bucket, key = get_profile_object_path(job_name=f"profile-test-raw-claim")




# Fetch the job results
results = databrew.describe_profile_job_run(jobName=f"profile-{get_job_name(path)}", runId=latest_run_id)
print(f"Job results: {results}")

# ----------------------------------------------------------------------------
# VALIDATE
# ----------------------------------------------------------------------------


Couldn't create crawler. Here's why: AlreadyExistsException: 618572314333:test-raw already exists


Crawler test-raw already exists.


Couldn't start crawler test-raw. Here's why: CrawlerRunningException: Crawler with name test-raw has already started


Crawler test-raw is already running.
Crawler test-raw is STOPPING.
	 10
Crawler test-raw is STOPPING.
	 21
Crawler test-raw is STOPPING.
	 32
Crawler test-raw is STOPPING.
	 43
Crawler test-raw is STOPPING.
	 53
Crawler test-raw is STOPPING.
	 64
Crawler test-raw is STOPPING.
	 75
Crawler test-raw is STOPPING.
	 85
Crawler test-raw is STOPPING.
	 96
Time to view result: 96.73635339736938


In [ ]:
print(json.dumps(glue.get_crawler(name),indent=4, sort_keys=True, default=str))

In [ ]:
import json

# use json dumps for result 
print(json.dumps(result, indent=4, sort_keys=True, default=str))

In [ ]:
import boto3

role = "arn:aws:iam::618572314333:role/service-role/AWSGlueServiceRole-ETL"
name = "boto3-dq-crawler"
database_name = "data-quality"
s3_target_path = "s3://project-lf/data_quality"

glue = GlueWrapper(boto3.client('glue', region_name='us-east-1'))

try:
    response = glue.create_crawler(name, role, database_name, '', s3_target_path)
    print(response)
except glue.glue_client.exceptions.AlreadyExistsException as e:
    print(e)
    
if glue.get_crawler(name) == None:
    print("Crawler does not exist.")
else:
    print("Crawler exists.")
    
try:
    response = glue.start_crawler(name)
    print(response)
except glue.glue_client.exceptions.CrawlerRunningException as e:
    print(e)

## Link to profile report as part of logging.
https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=landing-claimdb-policyholder&tab=profile-overview

`https://us-east-1.console.aws.amazon.com/databrew/home?region={region}#dataset-details?dataset={profile-job-name}&tab=profile-overview`